In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import sys
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np

In [4]:
def create_graph(model_path):
    """
    create_graph loads the inception model to memory, should be called before
    calling extract_features.
    model_path: path to inception model in protobuf form.
    """
    with gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [5]:
def extract_features(image_paths, verbose=False):
    """
    extract_features computed the inception bottleneck feature for a list of images
    image_paths: array of image path
    return: 2-d array in the shape of (len(image_paths), 2048)
    """
    feature_dimension = 2048
    features = np.empty((len(image_paths), feature_dimension))

    with tf.Session() as sess:
        flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for i, image_path in enumerate(image_paths):
            if verbose:
                print('Processing %s...' % (image_path))

            if not gfile.Exists(image_path):
                tf.logging.fatal('File does not exist %s', image)

            image_data = gfile.FastGFile(image_path, 'rb').read()
            feature = sess.run(flattened_tensor, {
                'DecodeJpeg/contents:0': image_data
            })
            features[i, :] = np.squeeze(feature)

    return features

In [3]:
model_path =os.path.expanduser('~/d_test/data/models/inception/classify_image_graph_def.pb')

In [7]:
create_graph(model_path)

In [39]:
labels = np.array([])
all_features = None
root_path = os.path.expanduser('~/d_test/data/pics/final_sets/cam_1_2/train')
for img_folder in os.listdir(root_path):
    print('processing dir {}'.format(img_folder))
    dummy = np.array([int(img_folder)],dtype='int')
    labels = np.append(labels, np.repeat(dummy,len(os.listdir(os.path.join(root_path, img_folder)))))
    img_paths = [os.path.join(root_path, img_folder, img) for img in os.listdir(os.path.join(root_path, img_folder))]
    features = extract_features(img_paths)
    if all_features is None:
        all_features = features
    else:
        all_features = np.concatenate((all_features, features))

processing dir 355
processing dir 563
processing dir 189
processing dir 708
processing dir 21
processing dir 14
processing dir 662
processing dir 499
processing dir 683
processing dir 111
processing dir 619
processing dir 100
processing dir 221
processing dir 649
processing dir 96
processing dir 585
processing dir 287
processing dir 194
processing dir 592
processing dir 137
processing dir 330
processing dir 180
processing dir 50
processing dir 35
processing dir 301
processing dir 524
processing dir 474
processing dir 423
processing dir 635
processing dir 140
processing dir 578
processing dir 386
processing dir 691
processing dir 316
processing dir 384
processing dir 353
processing dir 201
processing dir 610
processing dir 53
processing dir 268
processing dir 340
processing dir 449
processing dir 60
processing dir 233
processing dir 606
processing dir 359
processing dir 561
processing dir 334
processing dir 123
processing dir 655
processing dir 148
processing dir 209
processing dir 522


processing dir 13
processing dir 454
processing dir 707
processing dir 25
processing dir 564
processing dir 666
processing dir 338
processing dir 434
processing dir 75
processing dir 182
processing dir 511
processing dir 381
processing dir 317
processing dir 367
processing dir 523
processing dir 709
processing dir 377
processing dir 244
processing dir 466
processing dir 170
processing dir 272
processing dir 572
processing dir 387
processing dir 361
processing dir 615
processing dir 629
processing dir 266
processing dir 451
processing dir 569
processing dir 26
processing dir 85
processing dir 10
processing dir 8
processing dir 273
processing dir 69
processing dir 247
processing dir 653
processing dir 496
processing dir 293
processing dir 332
processing dir 479
processing dir 59
processing dir 196
processing dir 425
processing dir 279
processing dir 280
processing dir 692
processing dir 190
processing dir 516
processing dir 97
processing dir 43
processing dir 94
processing dir 327
proces

In [41]:
labels.shape

(23202,)

In [31]:
for v in labels:
    n_rep = len(os.listdir(os.path.join(root_path,str(int(v)))))
    dummy = np.array(v, dtype='int')
    right_labels = np.append(right_labels, np.repeat(dummy, n_rep))

In [11]:
saving_dir = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/cam_2')

In [12]:
np.save(os.path.join(saving_dir, 'cam_2_inception_features.npy'),features)
np.save(os.path.join(saving_dir, 'cam_2_inception_labels.npy'),labels)

In [24]:
features2.shape

(34, 2048)

In [45]:
tf.res

In [4]:
import sys
import os
sys.path.insert(0, '../verificator')

In [5]:
from verificator.feature_extractor import InceptionExtractor

In [6]:
inc_extr = InceptionExtractor(model_path)

In [7]:
root_path_2 = os.path.expanduser('~/d_test/data/pics/final_sets/clean_cam_2_thr_0.5/train')

In [8]:
labels, features = inc_extr.encode_from_dirs(root_path_2, verbose=True)

processing dir: 708
processing dir: 662
processing dir: 683
processing dir: 619
processing dir: 649
processing dir: 585
processing dir: 592
processing dir: 635
processing dir: 578
processing dir: 691
processing dir: 610
processing dir: 606
processing dir: 655
processing dir: 577
processing dir: 611
processing dir: 686
processing dir: 669
processing dir: 642
processing dir: 671
processing dir: 694
processing dir: 625
processing dir: 650
processing dir: 679
processing dir: 663
processing dir: 682
processing dir: 599
processing dir: 576
processing dir: 624
processing dir: 583
processing dir: 632
processing dir: 601
processing dir: 571
processing dir: 658
processing dir: 573
processing dir: 594
processing dir: 620
processing dir: 641
processing dir: 661
processing dir: 652
processing dir: 637
processing dir: 706
processing dir: 597
processing dir: 690
processing dir: 603
processing dir: 643
processing dir: 633
processing dir: 607
processing dir: 685
processing dir: 590
processing dir: 608


In [9]:
labels.shape

(3755,)

In [10]:
features.shape

(3755, 2048)

In [3]:
import sys
import os
sys.path.insert(0, '../verificator')

In [4]:
from verificator.tpe_predictor import TPEPredictor

Using TensorFlow backend.


In [5]:
import numpy as np

In [6]:
inception_predictor = TPEPredictor(2048,512)

../verificator/verificator/tpe_model/tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, weights=[array([[ ..., activation="linear", use_bias=False, input_dim=2048)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
../verificator/verificator/tpe_model/tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
/home/shareduser/.local/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
../verificator/verificator/tpe_model/tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inpu

In [7]:
inc_feat_1 = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/cam_1')
inc_feat_1_2 = os.path.expanduser('~/d_test/data/models/final_models/features/inception_features/cam_1_2')

In [8]:
features = np.load(os.path.join(inc_feat_1, 'cam_1_inception_features.npy'))
labels = np.load(os.path.join(inc_feat_1, 'cam_1_inception_labels.npy'))

In [9]:
w_saving_path_1 = os.path.expanduser('~/d_test/data/models/final_models/weights/inception/cam_1')
w_saving_path_1_2 = os.path.expanduser('~/d_test/data/models/final_models/weights/inception/cam_1_2')

In [10]:
mineer_inc, mindeer_inc = inception_predictor.train(
    20, 32, weights_saving_path=w_saving_path_1, weights_saving_name='cam_1_20_epoch_inception_features.h5',
    labels=labels, embeddings=features)

training tpe, number of train data (15921, 2048), number of dev data (3525, 2048), epochs 20
epoch: 0
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6916
min score: 0.4717029929161072, max score: 1.0000009536743164
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6700
min score: 0.296098530292511, max score: 1.0000009536743164
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6897
min score: 0.30053025484085083, max score: 1.000001072883606
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6783
min score: 0.11741795390844345, max score: 1.0000009536743164
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6943
min score: 0.2686372697353363, max score: 1.0000009536743164
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6845
min score: 0.19113148748874664, max score: 1.0000011920928955
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6831
min score: 0.09163792431354523, max score: 1.000

KeyboardInterrupt: 